In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime as dt

In [2]:
# Base URL to edit when iterating through dates
# John Hopkins Data
BASE_URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports_us/'

# States/Provinces to ignore - List is actually unused
excluded = ["Guam", "District of Columbia", "Grand Princess",
            "Diamond Princess","Northern Mariana Islands",
            "Puerto Rico", "Recovered", "American Samoa", "Virgin Islands"]

In [3]:
# Dict to convert full state name to initials for plotly compatibility
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI"
}

# Dict to convert UID to FIPS code for database coherency and vaccine graph functionality
uid_to_fips_dict = {
    84000001.0: 1.0,
    84000002.0: 2.0,
    16.0: 60.0,
    84000004.0: 4.0,
    84000005.0: 5.0,
    84000006.0: 6.0,
    84000008.0: 8.0,
    84000009.0: 9.0,
    84000010.0: 10.0,
    84088888.0: 88888.0,
    84000011.0: 11.0,
    84000012.0: 12.0,
    84000013.0: 13.0,
    84000014.0: 14.0,
    
    84099999.0: 99999.0,
    316.0: 66.0,
    84000016.0: 16.0,
    84000017.0: 17.0,
    84000018.0: 18.0,
    84000019.0: 19.0,
    84000020.0: 20.0,
    84000021.0: 21.0,
    84000022.0: 22.0,
    84000023.0: 23.0,
    84000024.0: 24.0,
    84000025.0: 25.0,
    84000026.0: 26.0,
    84000027.0: 27.0,
    84000028.0: 28.0,
    84000029.0: 29.0,
    84000030.0: 30.0,
    84000031.0: 31.0,
    84000032.0: 32.0,
    84000033.0: 33.0,
    84000034.0: 34.0,
    84000035.0: 35.0,
    84000036.0: 36.0,
    84000037.0: 37.0,
    84000038.0: 38.0,
    580.0: 69.0,
    84000039.0: 39.0,
    84000040.0: 40.0,
    84000041.0: 41.0,
    84000042.0: 42.0,
    630.0: 72.0,
    84000044.0: 44.0,
    84000045.0: 45.0,
    84000046.0: 46.0,
    84000047.0: 47.0,
    84000048.0: 48.0,
    84000049.0: 49.0,
    84000050.0: 50.0,
    850.0: 78.0,
    84000051.0: 51.0,
    84000053.0: 53.0,
    84000054.0: 54.0,
    84000055.0: 55.0,
    84000056.0: 56.0  
}

In [4]:
# Generating Date Ranges for CSV retrieval
dates = pd.date_range(start='04-12-2020', end='07-26-2022').strftime('%m-%d-%Y')
for date in dates[0:5]:
    print(date)
print()
print(f'Total Days: {len(dates)}')

04-12-2020
04-13-2020
04-14-2020
04-15-2020
04-16-2020

Total Days: 836


In [5]:
def generateSmallTables(dates = dates):
    for date in dates:
        url = BASE_URL + date + ".csv"
        print(url)
        df = pd.read_csv(url).fillna(0)
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

#generateSmallTables()

In [6]:
def editSmallTables(dates = dates):
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        df = pd.read_csv(path).fillna(0)
        df['Last_Update'] = date
        df.to_csv("data/SmallTables/" + date + ".csv", index=False)

#editSmallTables()

In [8]:
def generateBigTable(dates = dates):
    # Init main DataFrame
    allData = pd.DataFrame()
    # Iterate through days
    for date in dates:
        path = "data/SmallTables/" + date + ".csv"
        print(path)
        allData = pd.concat([allData, pd.read_csv(path).fillna(0)])
    # Pandas didn't like it when I tried anything but a boolean operator
    allData = allData[allData.Province_State != "Guam"]
    allData = allData[allData.Province_State != "District of Columbia"]
    allData = allData[allData.Province_State != "Grand Princess"]
    allData = allData[allData.Province_State != "Diamond Princess"]
    allData = allData[allData.Province_State != "Northern Mariana Islands"]
    allData = allData[allData.Province_State != "Puerto Rico"]
    allData = allData[allData.Province_State != "Recovered"]
    allData = allData[allData.Province_State != "American Samoa"]
    allData = allData[allData.Province_State != "Virgin Islands"]
    # Drop redundant columns
    allData = allData.drop(["ISO3", "FIPS", "Country_Region", "Date", "Recovered", "Active", "Incident_Rate",
                           "Total_Test_Results", "People_Hospitalized", "Case_Fatality_Ratio", "Hospitalization_Rate",
                           "People_Tested"], axis = 1)
    # Reorder so Primary Key is in front of DataFrame
    updated = allData.pop("Last_Update")
    allData["Province_State"] = allData["Province_State"].replace(us_state_to_abbrev)
    allData["UID"] = allData["UID"].replace(uid_to_fips_dict)
    fips = allData.pop("UID")
    allData.insert(0, "FIPS", fips)
    allData.insert(1, "Date", updated)
    # Write to CSV
    allData.to_csv("data/bigTable.csv", index=False)

generateBigTable()

data/SmallTables/04-12-2020.csv
data/SmallTables/04-13-2020.csv
data/SmallTables/04-14-2020.csv
data/SmallTables/04-15-2020.csv
data/SmallTables/04-16-2020.csv
data/SmallTables/04-17-2020.csv
data/SmallTables/04-18-2020.csv
data/SmallTables/04-19-2020.csv
data/SmallTables/04-20-2020.csv
data/SmallTables/04-21-2020.csv
data/SmallTables/04-22-2020.csv
data/SmallTables/04-23-2020.csv
data/SmallTables/04-24-2020.csv
data/SmallTables/04-25-2020.csv
data/SmallTables/04-26-2020.csv
data/SmallTables/04-27-2020.csv
data/SmallTables/04-28-2020.csv
data/SmallTables/04-29-2020.csv
data/SmallTables/04-30-2020.csv
data/SmallTables/05-01-2020.csv
data/SmallTables/05-02-2020.csv
data/SmallTables/05-03-2020.csv
data/SmallTables/05-04-2020.csv
data/SmallTables/05-05-2020.csv
data/SmallTables/05-06-2020.csv
data/SmallTables/05-07-2020.csv
data/SmallTables/05-08-2020.csv
data/SmallTables/05-09-2020.csv
data/SmallTables/05-10-2020.csv
data/SmallTables/05-11-2020.csv
data/SmallTables/05-12-2020.csv
data/Sma

data/SmallTables/01-27-2021.csv
data/SmallTables/01-28-2021.csv
data/SmallTables/01-29-2021.csv
data/SmallTables/01-30-2021.csv
data/SmallTables/01-31-2021.csv
data/SmallTables/02-01-2021.csv
data/SmallTables/02-02-2021.csv
data/SmallTables/02-03-2021.csv
data/SmallTables/02-04-2021.csv
data/SmallTables/02-05-2021.csv
data/SmallTables/02-06-2021.csv
data/SmallTables/02-07-2021.csv
data/SmallTables/02-08-2021.csv
data/SmallTables/02-09-2021.csv
data/SmallTables/02-10-2021.csv
data/SmallTables/02-11-2021.csv
data/SmallTables/02-12-2021.csv
data/SmallTables/02-13-2021.csv
data/SmallTables/02-14-2021.csv
data/SmallTables/02-15-2021.csv
data/SmallTables/02-16-2021.csv
data/SmallTables/02-17-2021.csv
data/SmallTables/02-18-2021.csv
data/SmallTables/02-19-2021.csv
data/SmallTables/02-20-2021.csv
data/SmallTables/02-21-2021.csv
data/SmallTables/02-22-2021.csv
data/SmallTables/02-23-2021.csv
data/SmallTables/02-24-2021.csv
data/SmallTables/02-25-2021.csv
data/SmallTables/02-26-2021.csv
data/Sma

data/SmallTables/10-22-2021.csv
data/SmallTables/10-23-2021.csv
data/SmallTables/10-24-2021.csv
data/SmallTables/10-25-2021.csv
data/SmallTables/10-26-2021.csv
data/SmallTables/10-27-2021.csv
data/SmallTables/10-28-2021.csv
data/SmallTables/10-29-2021.csv
data/SmallTables/10-30-2021.csv
data/SmallTables/10-31-2021.csv
data/SmallTables/11-01-2021.csv
data/SmallTables/11-02-2021.csv
data/SmallTables/11-03-2021.csv
data/SmallTables/11-04-2021.csv
data/SmallTables/11-05-2021.csv
data/SmallTables/11-06-2021.csv
data/SmallTables/11-07-2021.csv
data/SmallTables/11-08-2021.csv
data/SmallTables/11-09-2021.csv
data/SmallTables/11-10-2021.csv
data/SmallTables/11-11-2021.csv
data/SmallTables/11-12-2021.csv
data/SmallTables/11-13-2021.csv
data/SmallTables/11-14-2021.csv
data/SmallTables/11-15-2021.csv
data/SmallTables/11-16-2021.csv
data/SmallTables/11-17-2021.csv
data/SmallTables/11-18-2021.csv
data/SmallTables/11-19-2021.csv
data/SmallTables/11-20-2021.csv
data/SmallTables/11-21-2021.csv
data/Sma

data/SmallTables/07-09-2022.csv
data/SmallTables/07-10-2022.csv
data/SmallTables/07-11-2022.csv
data/SmallTables/07-12-2022.csv
data/SmallTables/07-13-2022.csv
data/SmallTables/07-14-2022.csv
data/SmallTables/07-15-2022.csv
data/SmallTables/07-16-2022.csv
data/SmallTables/07-17-2022.csv
data/SmallTables/07-18-2022.csv
data/SmallTables/07-19-2022.csv
data/SmallTables/07-20-2022.csv
data/SmallTables/07-21-2022.csv
data/SmallTables/07-22-2022.csv
data/SmallTables/07-23-2022.csv
data/SmallTables/07-24-2022.csv
data/SmallTables/07-25-2022.csv
data/SmallTables/07-26-2022.csv


In [9]:
testView = pd.read_csv("data/bigTable.csv")
print(testView.shape)
testView.head(100)

(41800, 9)


,FIPS,Date,Province_State,Lat,Long_,Confirmed,Deaths,Testing_Rate,Mortality_Rate
0,1.00,04-12-2020,AL,32.32,-86.90,3667,93,1265.08,2.54
1,2.00,04-12-2020,AK,61.37,-152.40,272,8,1344.71,2.94
2,4.00,04-12-2020,AZ,33.73,-111.43,3542,115,578.52,3.25
3,5.00,04-12-2020,AR,34.97,-92.37,1280,27,761.75,2.11
4,6.00,04-12-2020,CA,36.12,-119.68,22201,632,485.42,2.84
...,...,...,...,...,...,...,...,...,...
95,51.00,04-13-2020,VA,37.77,-78.17,5747,149,523.58,2.59
96,53.00,04-13-2020,WA,47.40,-121.49,10635,582,1242.51,5.47
97,54.00,04-13-2020,WV,38.49,-80.95,611,8,1258.88,1.31
98,55.00,04-13-2020,WI,44.27,-89.62,3428,154,0.00,4.49


In [ ]:
testView.describe()

In [ ]:
# Age and Sex Data
path = "data/SexAndAge/COVID-19_Death_Counts_by_Age_2020-2022.csv"
sex_age = pd.read_csv(path)
sex_age.head()

#sex_age = sex_age.drop(["Data as of", "Start Date", "End Date"], axis=1)
#sex_age.to_csv(path, index=False)
sex_age.head(200)

In [ ]:
def clean_vaccine(vaccine):
    vaccine = vaccine[vaccine.FIPS != 0]
    vaccine["Date"] = pd.to_datetime(vaccine["Date"]).dt.strftime('%m-%d-%Y')
    vaccine = vaccine.drop(["ID"], axis=1)
    #vaccine.to_csv(path, index=False)
    return vaccine

In [ ]:
# Load Vaccine Data
path = "data/vaccine2.csv"
vaccine = pd.read_csv(path, low_memory=False).fillna(0)
print(vaccine.shape)
vaccine.head(100)

In [ ]:
print(vaccine.shape)
vaccine.describe()

In [ ]:
def uid_to_fips(df):
    
    return df

In [13]:
location = testView[["FIPS", "Province_State", "Lat", "Long_"]]
location.to_csv("data/location.csv", index=False)
location.head()

,FIPS,Province_State,Lat,Long_
0,1.00,AL,32.32,-86.90
1,2.00,AK,61.37,-152.40
2,4.00,AZ,33.73,-111.43
3,5.00,AR,34.97,-92.37
4,6.00,CA,36.12,-119.68
